In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

def read_data(filename):
    data = {}
    i=0
    with open(filename, 'r') as file:
        for line in file:
            items = line.strip().split('\t')
            if i==0:
              i+=1
              index1= items[0]
              index2= items[1]
              index3= items[2]
              data[index1]=[]
              data[index2]=[]
              data[index3]=[]
            else:
              data[index1].append(items[0])
              data[index2].append(items[1])
              data[index3].append(items[2])

    return pd.DataFrame.from_dict(data)

In [ ]:
train_data = read_data("/content/drive/MyDrive/IR-Dataset/train_dataset.txt")
train_data

,user_id,item_id,rate
0,A2YKWYC3WQJX5J,B00106AC06,1
1,A2LXC5ZHHP0WXP,B00AE07BMQ,1
2,A3HLTHHLPKLRQA,B00AIQOKDY,1
3,A6N1DC5AMPLSK,B000F6RFX4,1
4,ALNFHVS3SC4FV,B0020122ZS,1
...,...,...,...
23033,A11I1I9QLMAM1A,B005TI7NPI,1
23034,AGFGY4EJ37VS2,B002PBHQP4,1
23035,A3MUSWDCTZINQZ,B009VGNYFM,1
23036,A1FWFCJU2G7TRA,B00D6EDGYE,1


In [ ]:
test_data = read_data("/content/drive/MyDrive/IR-Dataset/test_dataset.txt")
test_data

,user_id,item_id,rate
0,A35HOUWHAYZZN6,B00JF3RYPM,1
1,A1DGB6HY5C7LXU,B00KLA4INE,1
2,A3EPHBMU07LZ50,B00CBD0M8Y,1
3,A1WSQ0QRWQC7VI,B001FSK8SA,1
4,A2SZLNSI5KOQJT,B00D18ZDM8,1
...,...,...,...
5755,A16YU3GRGCD95S,B004D267RG,1
5756,A3HETHW2XD3DWH,B003BJZ1M4,1
5757,A23RZIU0N8K2KR,B00AE07CTI,1
5758,A3J356L2ADK05Z,B000AO2NXS,1


In [ ]:
len(train_data['user_id'].unique())

1340

In [ ]:
train_data['rate'].unique()

array(['1'], dtype=object)

In [ ]:
item_set = train_data['item_id'].unique()
user_set = train_data['user_id'].unique()


num_items = len(item_set)
num_users = len(user_set)

map_user = {}
map_item = {}

i=0
for item in item_set:
  map_item[item]=i
  i+=1

u = 0
for user in user_set:
  map_user[user]=u
  u+=1

In [ ]:
R={}
R = np.zeros((num_users, num_items))

for row in train_data.itertuples():
     i = map_item[row.item_id]
     u = map_user[row.user_id]
     R[u, i] = 1

In [ ]:
num_items

733

In [ ]:
num_users

1340

In [ ]:
import numpy as np

def dcg_score(score, k=None):
    if k is None:
        k = len(score)

    dcg=[]
    dcg.append(score[0])
    i=1
    while i < k:
      s = (score[i]/np.log2(i+1))
      dcg.append(s)
      i+=1

    return np.sum(dcg)
def ndcg_score(y_true, y_score, k=None):

    ideal_dcg = dcg_score(y_true, k)
    pred_dcg = dcg_score(y_score, k)

    if ideal_dcg == 0:
        return 0
    else:
        return pred_dcg / ideal_dcg

In [ ]:
import numpy as np

class MatrixFactorization:
    def __init__(self, num_items, num_users, num_factors, learning_rate, regularization_rate, num_iterations):
        """
        Initialize the Matrix Factorization model.

        Args:
            num_items (int): Number of items.
            num_users (int): Number of users.
            num_factors (int): Number of latent factors.
            learning_rate (float): Learning rate for gradient descent.
            regularization_rate (float): Regularization rate for L2 regularization.
            num_iterations (int): Number of iterations for training.
        """
        self.num_items = num_items
        self.num_users = num_users
        self.num_factors = num_factors
        self.learning_rate = learning_rate
        self.regularization_rate = regularization_rate
        self.num_iterations = num_iterations


        self.P = np.random.rand(self.num_users, self.num_factors)
        self.Q = np.random.rand(self.num_items, self.num_factors)

    def update_parameters(self, R):
        """
        Update the parameters Q and P using Stochastic Gradient Descent.

        Args:
            R (ndarray): Rating matrix.
        """

        self.R = R
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        for n in range(self.num_iterations):
            np.random.shuffle(self.samples)
            for u, i, r in self.samples:
              # Computer prediction and error
              prediction = self.predict_rating(i, u)
              grad = (r - prediction)
              # Update user and item latent feature matrices
              self.Q[i, :] -= self.learning_rate * (((-2) * grad * self.P[u, :]) + (2 * self.regularization_rate * self.Q[i, :] * (self.P[u, :] * self.P[u, :].T)))
              self.P[u, :] -= self.learning_rate * (((-2) * grad * self.Q[i, :]) + (2 * self.regularization_rate * self.P[u, :] * (self.Q[i, :] * self.Q[i, :].T)))
              # self.Q[i, :] -= self.learning_rate * (((-2) * grad * self.P[u, :]) + (2 * self.regularization_rate * self.Q[i, :] ))
              # self.P[u, :] -= self.learning_rate * (((-2) * grad * self.Q[i, :]) + (2 * self.regularization_rate * self.P[u, :] ))


    def train(self, R):
        """
        Train the Matrix Factorization model.

        Args:
            R (ndarray): Rating matrix.
        """
        self.update_parameters(R)

    def predict_rating(self, i, u):
        """
        Predict the rating for item i and user u.

        Args:
            i (int): Item index.
            u (int): User index.

        Returns:
            float: Predicted rating.
        """

        return self.Q[i, :].dot(self.P[u, :].T)

    def calculate_recall(self,users_list, groundTruth_list, topk=20):

        num_users = len(users_list)
        total_recall = 0

        for u in users_list:

           ground_truth = groundTruth_list[u]
           predicted_ratings = np.dot(self.Q , self.P[u, :].T)
           sorted_indices = np.argsort(predicted_ratings)[::-1]
           top_items = sorted_indices[:topk]
           intersection = set(top_items).intersection(ground_truth)
           # Recall = TruePositives / (TruePositives + FalseNegatives)
           recall = len(intersection) / len(ground_truth)
           total_recall += recall

        average_recall = total_recall / num_users

        return average_recall


    def calculate_ndcg(self,users_list, groundTruth_list, k=20):

        num_users = len(users_list)
        ndcg =0

        for u in users_list:

           ground_truth = groundTruth_list[u]

           predicted_ratings = np.dot(self.Q , self.P[u, :].T)
           sorted_indices = np.argsort(predicted_ratings)[::-1]
           top_items = sorted_indices[:k]

           pred_score = []
           true_score = []

           for t in top_items:
             if t in ground_truth:
              pred_score.append(1)
             else:
              pred_score.append(0)

           if(len(ground_truth) >= k):
              true_score = [1]*k
           else:
              true_score = [1]*len(ground_truth)+[0]*(k-len(ground_truth))

           ndcg+=ndcg_score(np.asarray(true_score), np.asarray(pred_score), k=k)

        average_ndcg = ndcg / num_users

        return average_ndcg

    def calculate_rank_correlation(self,users_list, groundTruth_list):

        rc =0
        num_users = len(users_list)
        s = (self.num_items * (pow(self.num_items,2)-1))

        for u in users_list:

           ground_truth = groundTruth_list[u]
           predicted_ratings = np.dot(self.Q , self.P[u, :].T)

           d2=0

           for i,val in enumerate(predicted_ratings):
             if i in ground_truth:
              val = val-1
             d2+=pow(val,2)

           rc+=1-((6*d2)/s)

        return rc / num_users


In [ ]:
num_factors = 64
regularization_rate = 0.001
num_iterations = 100
learning_rate = 0.01

In [ ]:
model = MatrixFactorization(num_items, num_users, num_factors, learning_rate, regularization_rate,
                            num_iterations)
model.train(R)

In [ ]:
item_index = 0
user_index = 0
model.predict_rating(item_index, user_index)

0.9988349431811991

In [ ]:
user_ids = test_data['user_id'].unique()

user_test = []
r_matrix = {}

for user in user_ids:
  user_test.append(map_user[user])
  r_matrix[map_user[user]] = []

for row in train_data.itertuples():
   i = map_item[row.item_id]
   u = map_user[row.user_id]

   r_matrix[u].append(i)

for row in test_data.itertuples():
   i = map_item[row.item_id]
   u = map_user[row.user_id]

   r_matrix[u].append(i)

In [ ]:
recall = model.calculate_recall(user_test, r_matrix)
ndcg = model.calculate_ndcg(user_test, r_matrix)
rank = model.calculate_rank_correlation(user_test, r_matrix)
print("Result:   recall={:.3f}\tndcg={:.3f}\trank_correlation = {:.5f}".format(recall,ndcg,rank))

Result:   recall=0.001	ndcg=0.002	rank_correlation = 0.99993
